# NB EMA

1) organizza il notebook in funzioni diverse, di modo che puoi ripetere le analisi su più di un dataset
2) esporta in csv il nuovo dataset, magari con le analisi fatte e pusha 
3) comincia a pensare alla normalizzazione 
4) vedi se questi cazzo di dati sono block sparse oppure no 


###  DS lab 2019

## Behavioural data

In [19]:
import pandas as pd
import numpy as np
import matplotlib as mpl

In [20]:
# import data into csv file

alldata_beh = pd.read_csv('data/Behavioral/AllData.csv')

/anaconda3/envs/dslab_1/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (66,68,80,81,82,83,84,95,96,97,98,99,110,111,112,113,114,125,126,127,128,129,140,141,142,144,155,156,157,159,932,933,934,935,936,937,938,939,940,941,942,943,944,945,946,947,948,949,950,951,952,953,954,955,956,957,958,959,960,961,962,963,964,965,966,967,968,969,970,971,972,973,974,975,976,977,978,979,980,981,982,983,984,985,986,987,988,989,990,991,992,993,994,995,996,997,998,999,1000,1001,1002,1003,1004,1005,1006,1007,1008,1009,1010,1011,1012,1013,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023,1024,1025,1026,1027,1028,1029,1030,1031,1032,1033,1034,1035,1036,1037,1038,1039,1040,1041,1042,1043,1044,1045,1046,1047,1048,1049,1050,1051,1052,1053,1054,1055,1056,1057,1058,1059,1060,1061,1062,1063,1064,1065,1066,1067,1068,1069,1070,1071,1072,1073,1074,1075,1076,1077,1078,1079,1080,1081,1082,1083,1084,1085,1086,1087,1088,1089,1090,1091,1092,1093,1094,1095,1096,1097,1098,1099,1100,1

### Nan percentages

In [3]:
def find_nan_percentages(dataframe): #determining NaN percentages in the dataframe, returns dict ith nan percentages
    
    nan_percentages = []    # [COLUMN_NAME, PERCENTAGE OF NANs]
    for column in dataframe: 
        n_elements = 0
        n_nan_elements = 0
        for row in dataframe.index: 
            elem = dataframe.loc[row, column]
            if pd.isna(elem)==True :
                n_nan_elements+=1
            n_elements+=1
        nan_percentages.append([column, round((n_nan_elements/n_elements),2)*100])
    nan_perc = dict(nan_percentages) #NaN percentages dict
    
    return nan_perc

'''
CHECKING ERRORS OR ANOMALIES IN DIAGNOSES

# check errors in diagnoses 12
err12 = 0
for row in alldata_beh.index: 
    if pd.isna(alldata_beh.loc[row,'DX_02_Cat'])==False and pd.isna(alldata_beh.loc[row,'DX_01_Cat'])==True: 
        err12+=1
    
# check errors in diagnoses 23
err23 = 0
for row in alldata_beh.index: 
    if pd.isna(alldata_beh.loc[row,'DX_03_Cat'])==False and pd.isna(alldata_beh.loc[row,'DX_02_Cat'])==True: 
        err12+=1

# check errors in diagnoses 34
err34 = 0
for row in alldata_beh.index: 
    if pd.isna(alldata_beh.loc[row,'DX_04_Cat'])==False and pd.isna(alldata_beh.loc[row,'DX_03_Cat'])==True: 
        err34+=1

print('errors : ',err12,err23,err34)

'''

"\nCHECKING ERRORS OR ANOMALIES IN DIAGNOSES\n\n# check errors in diagnoses 12\nerr12 = 0\nfor row in alldata_beh.index: \n    if pd.isna(alldata_beh.loc[row,'DX_02_Cat'])==False and pd.isna(alldata_beh.loc[row,'DX_01_Cat'])==True: \n        err12+=1\n    \n# check errors in diagnoses 23\nerr23 = 0\nfor row in alldata_beh.index: \n    if pd.isna(alldata_beh.loc[row,'DX_03_Cat'])==False and pd.isna(alldata_beh.loc[row,'DX_02_Cat'])==True: \n        err12+=1\n\n# check errors in diagnoses 34\nerr34 = 0\nfor row in alldata_beh.index: \n    if pd.isna(alldata_beh.loc[row,'DX_04_Cat'])==False and pd.isna(alldata_beh.loc[row,'DX_03_Cat'])==True: \n        err34+=1\n\nprint('errors : ',err12,err23,err34)\n\n"

In [ ]:
#isolate behavioral test data : no id info, nor diagnoses

beh_test_data = alldata_beh[ list(alldata_beh.loc[:,'ACE_01':]) ] 
beh_test_data

In [9]:
# find features within a threshold of NaN percentage, returns a list of the features

def find_frequent_features(dataframe, perc_threshold):
    nan_perc = find_nan_percentages(dataframe)
    freq_features = []   #features with less than 50% NaN 
    for feature in dataframe.columns: 
        if nan_perc.get(feature) < perc_threshold: 
            freq_features.append(feature)
    return freq_features
    

#btdata_f = beh_test_data.loc[:,freq_features]
        

In [24]:
# finding basic features : id, basics, diagnoses

basic_columns = alldata_beh[ list(alldata_beh.loc[:,:'Days_Baseline'])].columns  #columns with id, basic features and diagnoses
basic_features = []       #list with basic features 
for col in basic_columns: 
    basic_features.append(col)


In [ ]:
# filter alldata_beh with only basic and frequent features

allbdata_f = alldata_beh.loc[:, (basic_features+freq_features)]  #filtered alldata
allbdata_f  



In [ ]:
# exporting in csv 

# file = pd.to_csv('path')

### Nan percentages range in dataset

In [16]:
def nan_perc_ranges(dataframe):
    
    nan_percentages = find_nan_percentages(dataframe)
    features_perc_range = []   #list with percentages of NaN ranges associated with features
                               # [10, list_of_features] element means that labels associated are from 10% to 19.99% of NaN 
    for threshold in np.arange(10):
        feature_list = []
        for feature, perc in nan_percentages.items():
            if perc >= threshold*10.0 and perc <= (threshold+1)*10.0: 
                feature_list.append(feature)
        features_perc_range.append([threshold*10,feature_list])
        
    return features_perc_range

In [17]:
# printing NaN percentages ranges with associated features
features_perc_range = nan_perc_ranges(alldata_beh)
for perc,features in features_perc_range:
    print('number of features in', perc,'% - ', perc+10,'% range :', len(features))

number of features in 0 % -  10 % range : 56
number of features in 10 % -  20 % range : 673
number of features in 20 % -  30 % range : 365
number of features in 30 % -  40 % range : 422
number of features in 40 % -  50 % range : 314
number of features in 50 % -  60 % range : 160
number of features in 60 % -  70 % range : 1441
number of features in 70 % -  80 % range : 952
number of features in 80 % -  90 % range : 472
number of features in 90 % -  100 % range : 2614


## Matching columns with descriptions

In [21]:
# reading valid column names from the description files 

import glob

path = 'data/Behavioral/description' 
desc_files = glob.glob(path + "/*.xlsx")

varname_cols = []
desc_errors = 0
wrong_desc = []
processed_files = [] #list of processed files to avoid errors

for filename in desc_files:
    df = pd.read_excel(filename, skiprows=[0]) #index_col=None, header=0
    column_names = df.columns
    #print(column_names)
    if 'Variable Name' in column_names: 
        varname_cols.append(df.loc[:,'Variable Name'])
    elif 'Variable' in column_names: 
        varname_cols.append(df.loc[:,'Variable'])
    else:
        desc_errors += 1
        wrong_desc.append(filename)
    processed_files.append(filename)

print([file for file in processed_files])
        
'''
print ('description that do not contain a column that matches are :', desc_errors ,'\n',
       'the wrong descriptions are ',
        [d for d in wrong_desc])
        
FOUND THAT 'data/Behavioral/description/ESWAN.xlsx' RAISES PROBLEMS. NAIVE CORRECTION
'''

# fix issue with ESWAN 

path_ESWAN = 'data/Behavioral/description/ESWAN.xlsx'
df = pd.read_excel(path_ESWAN, skiprows=[0,1]) #index_col=None, header=0
#column_names = df.columns
varname_cols.append(df.loc[:,'Variable Name'])

# fix issue with nested directory 

path_inside_nested_dir = 'data/Behavioral/description/*'
desc_files_nested = glob.glob(path_inside_nested_dir + "/*.xlsx")
for filepath in desc_files_nested:
    df = pd.read_excel(filepath, skiprows=[0])
    varname_cols.append(df.loc[:,'Variable Name'])

# resulting list

print ('the resulting list has lenght', len (varname_cols))  #outputs 123. works. 




['data/Behavioral/description/ASSQ.xlsx', 'data/Behavioral/description/Quotient.xlsx', 'data/Behavioral/description/DigitSpan.xlsx', 'data/Behavioral/description/WISC.xlsx', 'data/Behavioral/description/ACE.xlsx', 'data/Behavioral/description/PAQ_A.xlsx', 'data/Behavioral/description/WHODAS_P.xlsx', 'data/Behavioral/description/SympChck.xlsx', 'data/Behavioral/description/YFAS.xlsx', 'data/Behavioral/description/EVT.xlsx', 'data/Behavioral/description/PCIAT.xlsx', 'data/Behavioral/description/SDQ.xlsx', 'data/Behavioral/description/WHODAS_SR.xlsx', 'data/Behavioral/description/WASI.xlsx', 'data/Behavioral/description/TOWRE.xlsx', 'data/Behavioral/description/NIH_Scores.xlsx', 'data/Behavioral/description/ACE_Flanker.xlsx', 'data/Behavioral/description/CDI_P.xlsx', 'data/Behavioral/description/CTOPP.xlsx', 'data/Behavioral/description/SWAN.xlsx', 'data/Behavioral/description/RANRAS.xlsx', 'data/Behavioral/description/PreInt_Demos_Home.xlsx', 'data/Behavioral/description/CGAS.xlsx', 'dat

In [22]:
varnames_df = pd.DataFrame(np.hstack([i.values for i in varname_cols])) 
varnames_df = varnames_df.dropna()    #drop nan
varnames = varnames_df.values  # numpy array with variable names from descriptions
varnames_list = [elem[0] for elem in varnames.tolist()]  #list with variable names from descriptions

In [25]:
beh_data_matched = alldata_beh.copy(deep=True)   # dataframe with behavioural data keeping only basic columns and the ones mentioned in the decriptions
for column in beh_data_matched.columns: 
    if column not in varnames_list and column not in basic_features:
        beh_data_matched.drop(columns=column, inplace=True)

In [26]:
beh_data_matched

,Anonymized.ID,Subject.Type,Visit,Days.since.enrollment,EID,START_DATE,Patient_ID,Enrollment.Year,Enrollment.Season,Sex,...,YSR_TP,YSR_TP_T,YSR_Ext,YSR_Ext_T,YSR_Int,YSR_Int_T,YSR_OP,YSR_C,YSR_Total,YSR_Total_T
0,A00078864,UNKNOWN,V1,0,NDARYM832PX3,1901-01-01T07:00:00Z,NDARYM832PX3,2015,Spring,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,A00078865,UNKNOWN,V1,0,NDARNJ687DMC,1901-01-01T07:00:00Z,NDARNJ687DMC,2015,Spring,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,A00078866,UNKNOWN,V1,0,NDARRM363BXZ,1901-01-01T07:00:00Z,NDARRM363BXZ,2015,Spring,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,A00078867,UNKNOWN,V1,0,NDARUW586LLL,1901-01-01T07:00:00Z,NDARUW586LLL,2015,Spring,1,...,15.0,75.0,16.0,60.0,29.0,68.0,8.0,40.0,85.0,68.0
4,A00078868,UNKNOWN,V1,0,NDARDC298NW4,1901-01-01T07:00:00Z,NDARDC298NW4,2015,Spring,0,...,9.0,67.0,10.0,52.0,26.0,72.0,11.0,34.0,70.0,67.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2091,A00093557,UNKNOWN,V1,0,NDARYZ986HEW,1901-01-01T07:00:00Z,NDARYZ986HEW,2018,Summer,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2092,A00093558,UNKNOWN,V1,0,NDARPM572ZZV,1901-01-01T07:00:00Z,NDARPM572ZZV,2018,Summer,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2093,A00093559,UNKNOWN,V1,0,NDARYA955CY1,1901-01-01T07:00:00Z,NDARYA955CY1,2018,Fall,0,...,6.0,60.0,11.0,53.0,13.0,59.0,3.0,23.0,47.0,58.0
2094,A00093560,UNKNOWN,V1,0,NDARRT283RHB,1901-01-01T07:00:00Z,NDARRT283RHB,2018,Fall,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
#converting to strings as the original data is, before exporting in csv
beh_data_matched = beh_data_matched.astype(str)

In [31]:
# exporting in csv behavioral data matched
file = beh_data_matched.to_csv('data/Behavioral/behdata_matched_ema.csv')